In [3]:
import soccerdata as sd
import pandas as pd
import numpy as np

In [2]:
temporadas = ["17-18", "18-19", "19-20","20-21", "21-22"]
fbref = sd.FBref(leagues="Big 5 European Leagues Combined", seasons=temporadas)

jogadores_standard_stats = fbref.read_player_season_stats(stat_type="standard")
jogadores_shooting_stats = fbref.read_player_season_stats(stat_type="shooting")
jogadores_passing_stats = fbref.read_player_season_stats(stat_type="passing")
jogadores_passing_type_stats = fbref.read_player_season_stats(stat_type="passing_types")
jogadores_goal_shot_creation_stats = fbref.read_player_season_stats(stat_type="goal_shot_creation")
jogadores_defense_stats = fbref.read_player_season_stats(stat_type="defense")
jogadores_possession_stats = fbref.read_player_season_stats(stat_type="possession")
jogadores_playing_time_stats = fbref.read_player_season_stats(stat_type="playing_time")
jogadores_miscellanous_type_stats = fbref.read_player_season_stats(stat_type="misc")


[07/01/23 17:03:37] INFO     Saving cached data to /home/gabrielmmf/soccerdata/data/FBref             ]8;id=420874;file:///home/gabrielmmf/anaconda3/envs/env1/lib/python3.9/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=704013;file:///home/gabrielmmf/anaconda3/envs/env1/lib/python3.9/site-packages/soccerdata/_common.py#89\89]8;;\

In [34]:
def getFbrefFromTransfers(stats, transfers_players, name):
    df = stats

    df = df.reset_index()

    multiindex = df.columns
    index = ['_'.join(level) if level[1] != '' else level[0] for level in multiindex]

    df.set_axis(index, axis=1, inplace=True)
    df["season"] = df["season"].replace({"1718": "2017",
                                     "1819": "2018",
                                     "1920": "2019",
                                     "2021": "2020",
                                     "2122": "2021"
                                     })

    df["nextSeason"] = df["season"].replace({"2017": "2018",
                                            "2018": "2019",
                                            "2019": "2020",
                                            "2020": "2021",
                                            "2021": "2022"
                                            })
    
    df['season'] = df['season'].astype('int')
    df['nextSeason'] = df['nextSeason'].astype('int')

    merged = pd.merge(transfers_players, df, left_on=['playerName', 'season'], right_on=['player', 'nextSeason'], how='left')
    del merged['season_y']
    merged = merged.rename(columns={'season_x': 'season'})
    merged = merged.dropna(axis=0, how="all", subset=['nextSeason'])

    merged.to_csv('data/transfers_stats/'+ name, index=False)

In [17]:
def getTransfersPlayers(league):
    players = pd.read_csv('data/players/'+league+'_players.csv')
    transfers = pd.read_csv('data/transfers/'+ league + '_transfers.csv')
    transfers_players = pd.merge(transfers, players[['playerLink', 'currAge', 'height', 'foot']], on="playerLink", how="left")
    transfers_players = transfers_players[transfers_players['season'] >= 2018]
    return transfers_players

In [18]:
pl_transfers_players = getTransfersPlayers('pl')
ll_transfers_players = getTransfersPlayers('ll')
sa_transfers_players = getTransfersPlayers('sa')
bl_transfers_players = getTransfersPlayers('bl')
l1_transfers_players = getTransfersPlayers('l1')

In [36]:
getFbrefFromTransfers(jogadores_standard_stats, pl_transfers_players, 'pl_std_stats.csv')

/tmp/ipykernel_312607/237810228.py:9: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(index, axis=1, inplace=True)


In [ ]:
getFbrefFromTransfers(jogadores_standard_stats, ll_transfers_players, 'll_std_stats.csv')

In [ ]:
getFbrefFromTransfers(jogadores_standard_stats, bl_transfers_players, 'bl_std_stats.csv')

In [ ]:
getFbrefFromTransfers(jogadores_standard_stats, sa_transfers_players, 'sa_std_stats.csv')

In [ ]:
getFbrefFromTransfers(jogadores_standard_stats, l1_transfers_players, 'l1_std_stats.csv')